# Tensorflow Estimators

In [1]:
import tensorflow as tf
import pandas as pd

In [2]:
df = pd.read_csv('../data/iris.csv')

In [3]:
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [26]:
X = df.drop('species', axis='columns')
y = df['species']

column_names = df.columns[:-1]
column_names

Index(['sepal_length', 'sepal_width', 'petal_length', 'petal_width'], dtype='object')

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [7]:
y_train.head()

60     versicolor
116     virginica
144     virginica
119     virginica
108     virginica
Name: species, dtype: object

In [8]:
labels = ['setosa', 'versicolor', 'virginica']

In [9]:
feature_columns = [
    tf.feature_column.numeric_column(key=key)
    for key in X_train.columns
]

In [10]:
feature_columns

[_NumericColumn(key='sepal_length', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='sepal_width', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='petal_length', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='petal_width', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

## Canned Estimators

In [11]:
rm -rf ./models/iris/

In [12]:
model = tf.estimator.DNNClassifier(
    hidden_units=[10, 10], 
    feature_columns=feature_columns,
    model_dir='./models/iris/DNN/',
    n_classes=3,
    label_vocabulary=labels,
    optimizer='Adam')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './models/iris/DNN/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f2a6d0ac4a8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [13]:
batch_size=32
train_steps=3000

### Input function

In [14]:
def input_fn(features, labels, batch_size, shuffle=True, repeat=None):
    feature_dict = dict(features)

    if labels is None:
        inputs = feature_dict
    else:
        inputs = (feature_dict, labels)
        
    dataset = tf.data.Dataset.from_tensor_slices(inputs)
    
    if shuffle:
        dataset = dataset.shuffle(1000)
    
    dataset = dataset.repeat(repeat)

    return dataset.batch(batch_size)

In [15]:
def train_input_fn():
    return input_fn(X_train, y_train, batch_size)

### Model Training

In [16]:
model.train(
    input_fn=train_input_fn,
    steps=3000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ./models/iris/DNN/model.ckpt.
INFO:tensorflow:loss = 35.619354, step = 0
INFO:tensorflow:global_step/sec: 231.432
INFO:tensorflow:loss = 0.6928652, step = 100 (0.434 sec)
INFO:tensorflow:global_step/sec: 289.623
INFO:tensorflow:loss = 2.7589676, step = 200 (0.347 sec)
INFO:tensorflow:global_step/sec: 288.208
INFO:tensorflow:loss = 5.418964, step = 300 (0.345 sec)
INFO:tensorflow:global_step/sec: 295.618
INFO:tensorflow:loss = 1.6653299, step = 400 (0.339 sec)
INFO:tensorflow:global_step/sec: 273.758
INFO:tensorflow:loss = 4.0081143, step = 500 (0.366 sec)
INFO:tensorflow:global_step/sec: 293.988
INFO:tensorflow:loss = 0.31819138, step = 600 (0.341 sec)
INFO:tensorflow:global_step/sec: 292.086
INFO:tensorflow

### Evaluate model on train and test data

In [17]:
train_eval_result = model.evaluate(
    input_fn=lambda:input_fn(X_train, y_train,
                             batch_size,
                             shuffle=False,
                             repeat=1))

test_eval_result = model.evaluate(
    input_fn=lambda:input_fn(X_test, y_test,
                             batch_size,
                             shuffle=False,
                             repeat=1))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-09-21-17:13:04
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./models/iris/DNN/model.ckpt-3000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-09-21-17:13:04
INFO:tensorflow:Saving dict for global step 3000: accuracy = 0.9809524, average_loss = 0.03287781, global_step = 3000, loss = 0.8630425
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 3000: ./models/iris/DNN/model.ckpt-3000
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-09-21-17:13:05
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./models/iris/DNN/model.ckpt-3000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-

In [18]:
train_eval_result

{'accuracy': 0.9809524,
 'average_loss': 0.03287781,
 'loss': 0.8630425,
 'global_step': 3000}

In [19]:
test_eval_result

{'accuracy': 0.9777778,
 'average_loss': 0.13602027,
 'loss': 3.0604563,
 'global_step': 3000}

### Predict on new data

In [20]:
predict_x = {
    'sepal_length': [5.1, 5.9, 6.9],
    'sepal_width': [3.3, 3.0, 3.1],
    'petal_length': [1.7, 4.2, 5.4],
    'petal_width': [0.5, 1.5, 2.1],
}

In [21]:
expected = ['setosa', 'versicolor', 'virginica']

In [22]:
predictions = model.predict(
    input_fn=lambda:input_fn(predict_x,
                             labels=None,
                             batch_size=batch_size,
                             shuffle=False,
                             repeat=1))

In [23]:
predictions

<generator object Estimator.predict at 0x7f2a4d4158e0>

In [24]:
template = ('\nPrediction is "{}", prob: {:.1f}%, expected "{}"')

for pred_dict, expec in zip(predictions, expected):
    
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]
    pred_label = labels[class_id]

    print(template.format(pred_label, 100 * probability, expec))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./models/iris/DNN/model.ckpt-3000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.

Prediction is "setosa", prob: 100.0%, expected "setosa"

Prediction is "versicolor", prob: 100.0%, expected "versicolor"

Prediction is "virginica", prob: 99.9%, expected "virginica"


In [25]:
pred_dict

{'logits': array([-18.517982  ,  -0.11734152,   7.4038777 ], dtype=float32),
 'probabilities': array([5.5213373e-12, 5.4117898e-04, 9.9945885e-01], dtype=float32),
 'class_ids': array([2]),
 'classes': array([b'virginica'], dtype=object)}

## Exercise 1

The `tf.estimator` module offers 2 additional canned classifiers:

- `tf.estimator.BaselineClassifier`
- `tf.estimator.LinearClassifier`

What do they do?

Try substituting them in the model definition and re-run the trainin, evaluation and prediction steps.
Is the performance better or worse?

Note: you can set the logging level of tensorflow with `tf.logging.set_verbosity`

In [33]:
model = tf.estimator.BaselineClassifier(
    model_dir='./models/iris/Baseline/',
    n_classes=3,
    label_vocabulary=labels,
    optimizer='Adam')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './models/iris/Baseline/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f2a6c2c80b8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [34]:
model.train(input_fn=train_input_fn, steps=3000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ./models/iris/Baseline/model.ckpt.
INFO:tensorflow:loss = 35.155594, step = 0
INFO:tensorflow:global_step/sec: 245.114
INFO:tensorflow:loss = 34.684048, step = 100 (0.413 sec)
INFO:tensorflow:global_step/sec: 273.619
INFO:tensorflow:loss = 34.897446, step = 200 (0.362 sec)
INFO:tensorflow:global_step/sec: 319.096
INFO:tensorflow:loss = 36.704643, step = 300 (0.313 sec)
INFO:tensorflow:global_step/sec: 326.483
INFO:tensorflow:loss = 35.619537, step = 400 (0.307 sec)
INFO:tensorflow:global_step/sec: 322.908
INFO:tensorflow:loss = 35.087463, step = 500 (0.311 sec)
INFO:tensorflow:global_step/sec: 312.214
INFO:tensorflow:loss = 35.78099, step = 600 (0.319 sec)
INFO:tensorflow:global_step/sec: 318.009
INFO:tensor

In [35]:
train_eval_result = model.evaluate(input_fn=lambda:input_fn(X_train, y_train, batch_size, shuffle=False, repeat=1))
test_eval_result = model.evaluate(input_fn=lambda:input_fn(X_test, y_test, batch_size, shuffle=False, repeat=1))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-09-21-17:26:54
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./models/iris/Baseline/model.ckpt-3000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-09-21-17:26:54
INFO:tensorflow:Saving dict for global step 3000: accuracy = 0.37142858, average_loss = 1.0999277, global_step = 3000, loss = 28.8731
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 3000: ./models/iris/Baseline/model.ckpt-3000
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-09-21-17:26:55
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./models/iris/Baseline/model.ckpt-3000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evalua

In [36]:
train_eval_result

{'accuracy': 0.37142858,
 'average_loss': 1.0999277,
 'loss': 28.8731,
 'global_step': 3000}

In [37]:
test_eval_result

{'accuracy': 0.24444444,
 'average_loss': 1.1453903,
 'loss': 25.77128,
 'global_step': 3000}

In [41]:
model = tf.estimator.LinearClassifier(
    feature_columns=feature_columns,
    model_dir='./models/iris/Linear/',
    n_classes=3,
    label_vocabulary=labels,
    optimizer='Adam')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './models/iris/Linear/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f2a4d3b5a20>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [42]:
model.train(input_fn=train_input_fn, steps=3000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ./models/iris/Linear/model.ckpt.
INFO:tensorflow:loss = 35.155594, step = 0
INFO:tensorflow:global_step/sec: 202.276
INFO:tensorflow:loss = 3.5206366, step = 100 (0.496 sec)
INFO:tensorflow:global_step/sec: 235.441
INFO:tensorflow:loss = 2.9380765, step = 200 (0.425 sec)
INFO:tensorflow:global_step/sec: 219.536
INFO:tensorflow:loss = 2.687008, step = 300 (0.455 sec)
INFO:tensorflow:global_step/sec: 235.655
INFO:tensorflow:loss = 1.3233175, step = 400 (0.425 sec)
INFO:tensorflow:global_step/sec: 244.79
INFO:tensorflow:loss = 0.5299399, step = 500 (0.408 sec)
INFO:tensorflow:global_step/sec: 245.632
INFO:tensorflow:loss = 3.3368106, step = 600 (0.409 sec)
INFO:tensorflow:global_step/sec: 249.538
INFO:tensorflo

In [43]:
train_eval_result = model.evaluate(input_fn=lambda:input_fn(X_train, y_train, batch_size, shuffle=False, repeat=1))
test_eval_result = model.evaluate(input_fn=lambda:input_fn(X_test, y_test, batch_size, shuffle=False, repeat=1))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-09-21-17:30:39
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./models/iris/Linear/model.ckpt-3000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-09-21-17:30:40
INFO:tensorflow:Saving dict for global step 3000: accuracy = 1.0, average_loss = 0.026492368, global_step = 3000, loss = 0.6954247
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 3000: ./models/iris/Linear/model.ckpt-3000
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-09-21-17:30:40
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./models/iris/Linear/model.ckpt-3000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2

In [44]:
train_eval_result

{'accuracy': 1.0,
 'average_loss': 0.026492368,
 'loss': 0.6954247,
 'global_step': 3000}

In [45]:
test_eval_result

{'accuracy': 0.9777778,
 'average_loss': 0.10586121,
 'loss': 2.3818772,
 'global_step': 3000}

## Exercise 2

Visualize the training results with tensorboard:

- in a terminal, from the notebook folder, run tensorboard with `tensorboard --logdir=models`
- open the browser at [http://localhost:6006](http://localhost:6006)

